## Modelo: **Llama 3.1**
---

In [1]:
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3.1:latest"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

### Extraer texto del archivo PDF con el módulo `PyPDFLoader` de ***LangChain*** 

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/manual-lpcr.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'docs/manual-lpcr.pdf', 'page': 0}, page_content='MANUAL DEUSUARIO\nBÁSCULA ELECTRÓNICA\nREV0ANTES DE OPERAR SU BÁSCULA, FAVOR DE LEER \nESTE MANUAL CUIDADOSAMENTE.\nCODIGO: 13100930\nSeries L -PCR'),
 Document(metadata={'source': 'docs/manual-lpcr.pdf', 'page': 1}, page_content='2 Bascula electrónicaÍNDICE\nÍndice\nAdvertencias Importantes\nIntroducción\nInstrucciones de instalación\nNombre y localización de piezas    \nTeclado\nVisor/Pantalla\nDescripción de las operaciones\nCómo pesar un producto\nComo utilizar la tecla TARA\nCómo introducir precios     \nFunción PRECIO FIJO\nCómo usar la Referencia a CERO  \nSuma de productos\nVisualizar total de la venta al momento  \nVisualizar Venta Total \nBorrar Venta Total \nGrabar las memorias PLU`s\nRecordar una memoria\nCalcular Cambio \nFunción de apagado automático\nIndicador de batería baja \nUso de la batería\nOtros mensaje importantes de la bascula\nMantenimiento y limpieza\nGuía para la localización de a

In [3]:
from langchain.prompts import PromptTemplate

template = """
Responde la pregunta basado en el contexto de abajo. Si no puedes
responder la pregunta, responde "No lo se"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Esto es un contexto", question="Esto es una pregunta"))


Responde la pregunta basado en el contexto de abajo. Si no puedes
responder la pregunta, responde "No lo se"

Context: Esto es un contexto

Question: Esto es una pregunta



In [4]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

/Users/ivanlee/llama-rag-demo/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
#from operator import itemgetter

#chain = (
#    {
#        "context": itemgetter("question") | retriever,
#        "question": itemgetter("question")
#    } 
#    | prompt
#    | model 
#)

In [6]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
)

In [7]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(f"Pregunta: {question}")
    print(f"Respuesta: {chain.invoke({'question': question})}")
    print()

Pregunta: Como peso un producto?
Respuesta: La respuesta es: "Encienda la báscula presionando la tecla ON y luego coloque el producto en la plataforma. Luego, el visor/pantalla mostrará ceros y en ese momento la báscula está lista para pesar. Después de colocar el producto, el indicador de peso mostrará el peso exacto del producto."

Pregunta: Como se introduce el precio por kilogramo?
Respuesta: No lo sé

Pregunta: Como puedo borrar la venta total?
Respuesta: Según el manual de instrucciones de la Báscula Electrónica, puedes borrar la venta total presionando una tecla específica. La respuesta correcta es:

Visualizar Venta Total \nBorrar Venta Total 

Por lo tanto, la respuesta a tu pregunta es: "Presiona el botón 'Borrar Venta Total'".



### Extraer texto en *markdown* del archivo PDF con [Marker](https://github.com/VikParuchuri/marker])

- Para convertir un solo archivo (desde la terminal):

    `marker_single /path/to/file.pdf /path/to/output/folder --batch_multiplier 2 --max_pages 10`

- Para convertir multiples archivos:

    `marker /path/to/input/folder /path/to/output/folder --workers 4 --max 10 --min_length 10000`
---

***Nota:*** *Recomendado utilizar GPU (CUDA o MPS) y 16 GB de RAM*


In [8]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

markdown_path = "docs/manual-lpcr/manual-lpcr.md" 
md_loader = UnstructuredMarkdownLoader(markdown_path)

md_pages = md_loader.load_and_split()
md_pages 

[Document(metadata={'source': 'docs/manual-lpcr/manual-lpcr.md'}, page_content='MANUAL DE USUARIO BÁSCULA ELECTRÓNICA\n\nSeries L-PCR antes de operar su báscula, favor de leer ESTE MANUAL CUIDADOSAMENTE.\n\nÍndice\n\n2 3 5 5 6 7 8 9 9 9 10 11 11 11 12 12 12 13 13 13 14 14 14 15 16 17 18 Índice\n\nAdvertencias Importantes Introducción Instrucciones de instalación Nombre y localización de piezas Teclado Visor/Pantalla Descripción de las operaciones Cómo pesar un producto Como utilizar la tecla TARA Cómo introducir precios Función PRECIO FIJO Cómo usar la Referencia a CERO Suma de productos Visualizar total de la venta al momento Visualizar Venta Total Borrar Venta Total Grabar las memorias PLU`s Recordar una memoria Calcular Cambio Función de apagado automático Indicador de batería baja Uso de la batería Otros mensaje importantes de la bascula Mantenimiento y limpieza Guía para la localización de averías Especificaciones técnicas\n\nLa Batería Debe Ser Cargada Por 10 Hrs. Antes De Usar L

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    md_pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

In [10]:
md_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
)

In [11]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(f"Pregunta: {question}")
    print(f"Respuesta: {md_chain.invoke({'question': question})}")
    print()

Pregunta: Como peso un producto?
Respuesta: La respuesta a esta pregunta es:

"Para usar la báscula, primero asegúrate de que esté en buenas condiciones y nivelada. Conecta el adaptador CA/CC si lo utilizas y enciende la bascula. Luego, coloca el producto sobre el plato de la báscula y asegúrate de que no esté maltratado o golpeado durante el transporte. Si la bascula muestra 'HELP', puede ser debido a un daño irreversible causado por el transporte y debes dirigirte al distribuidor autorizado. Luego, simplemente lee la cantidad en el visor/pantalla para obtener el peso del producto."

Nota: Esta respuesta se basa en el texto proporcionado en el manual de instrucciones de la báscula electrónica.

Pregunta: Como se introduce el precio por kilogramo?
Respuesta: La respuesta es mediante la tecla "+" de la báscula, donde se selecciona "Precio Max/ kg" y luego se ingresa el monto máximo permitido.

Pregunta: Como puedo borrar la venta total?
Respuesta: La respuesta a la pregunta es:

"Presio

## Modelo: GPT-4o
---